In [11]:
import arcpy

# -------------------------------------------------------------------
# User-defined parameters
# -------------------------------------------------------------------
parcels_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Outputs\TaxPlat_No_P1\TaxPlat_No_P1.shp"
osds_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\OSDS\OahuOSDS\On-site_Sewage_Disposal_Systems_-_Oahu.shp"
footprints_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Building_Footprint_Centerpoints\Building_Footprint_CP_Projected\Building_Footprint_CP_Projected.shp"
output_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Outputs\Parcels_with_Counts.shp"

# -------------------------------------------------------------------
# Output field names
# -------------------------------------------------------------------
osds_count_field = "OSDS_Ct"
footprint_count_field = "Ftprt_Ct"

# -------------------------------------------------------------------
# 1. Add Fields for Counts in the Parcel Feature Class
# -------------------------------------------------------------------
print("Adding count fields to parcels...")
arcpy.env.overwriteOutput = True

if osds_count_field not in [f.name for f in arcpy.ListFields(parcels_fc)]:
    arcpy.management.AddField(parcels_fc, osds_count_field, "LONG")
if footprint_count_field not in [f.name for f in arcpy.ListFields(parcels_fc)]:
    arcpy.management.AddField(parcels_fc, footprint_count_field, "LONG")

# -------------------------------------------------------------------
# 2. Spatial Join for OSDS Counts (Points within Polygons)
# -------------------------------------------------------------------
print("Counting OSDS per parcel...")
osds_join_output = "in_memory\\osds_join"
arcpy.analysis.SpatialJoin(
    target_features=parcels_fc,
    join_features=osds_fc,
    out_feature_class=osds_join_output,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="CONTAINS"
)

# -------------------------------------------------------------------
# 3. Spatial Join for Footprint Counts (Points within Polygons)
# -------------------------------------------------------------------
print("Counting building footprints per parcel...")
footprints_join_output = "in_memory\\footprint_join"
arcpy.analysis.SpatialJoin(
    target_features=osds_join_output,
    join_features=footprints_fc,
    out_feature_class=footprints_join_output,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="CONTAINS"
)

# -------------------------------------------------------------------
# 4. Update the Attribute Table with the Counts
# -------------------------------------------------------------------
print("Updating parcel attribute table with counts...")
with arcpy.da.UpdateCursor(footprints_join_output, ["Join_Count", osds_count_field, footprint_count_field]) as cursor:
    for row in cursor:
        osds_count = row[0]  # Count of OSDS points from spatial join
        footprint_count = row[1]  # Count of footprint points from spatial join
        row[1] = osds_count if osds_count else 0  # Default to 0 if null
        row[2] = footprint_count if footprint_count else 0
        cursor.updateRow(row)

# -------------------------------------------------------------------
# 5. Save Final Output
# -------------------------------------------------------------------
print("Saving final output...")
arcpy.management.CopyFeatures(footprints_join_output, output_fc)

print(f"Counts added. Final output saved at {output_fc}")


Adding count fields to parcels...
Counting OSDS per parcel...
Counting building footprints per parcel...
Updating parcel attribute table with counts...
Saving final output...
Counts added. Final output saved at C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Outputs\Parcels_with_Counts.shp
